# NEUFEP-ME study: Data setup workflow

This notebook contains the workflow required to convert all raw data into the formats, types and structures necessary for the main data processing pipeline. Thus, getting data ready to be processed.

## Study overview

The main purpose of the study is to investigate methods to improve the quality of real-time functional magnetic resonance imaging (fMRI) data. These improvements are for future applications in real-time fMRI neurofeedback, a method where participants are presented with visual feedback of their brain activity while they are inside the MRI scanner, and then asked to regulate the level of feedback. We have developed real-time multi-echo EPI acquisition sequences and processing methods, and this study aims to collect data from volunteers in order to validate these new methods. No neurofeedback is provided.

```
The scan session will consist of a number of scan sequences, some of which will require you to look at pictures or perform a task, and others just to lie still in the scanner. The total time in the scanner will be around 1 hour, with a break in the middle. Volunteers have to be right-handed, healthy and should have no history of or be under current treatment for psychiatric / neurological conditions. If this inclusion criteria do not fit you, you will unfortunately not be able to participate in the study. Other reasons for not being able to participate include being pregnant or having metal implants.
```

## Data

Per subject, all of the following data were collected during one scan session (all functional scans are multi-echo EPI):

| Nr | Name  | Scan Type | Description | Format |
| :--- | :--- | :--- | :--- | :--- |
| 1 | T1w | Anatomical | Standard T1-weighted sequence | NIfTI |
| 2 | run1_BOLD_rest | Functional | Resting state | PAR/REC, XML/REC, DICOM |
| 3 | run1_BOLD_task1 | Functional | Motor - finger tapping | PAR/REC, XML/REC, DICOM |
| 4 | run1_BOLD_task2 | Functional | Emotion - shape/face matching | PAR/REC, XML/REC, DICOM |
| 5 | run2_BOLD_rest | Functional | Resting state | PAR/REC, XML/REC, DICOM |
| 6 | run2_BOLD_task1 | Functional | Motor mental - imagined finger tapping | PAR/REC, XML/REC, DICOM |
| 7 | run2_BOLD_task2 | Functional | Emotion mental - recalling emotional memories | PAR/REC, XML/REC, DICOM |
| 8 | Stimulus timing | Peripheral measure | Stimulus and response timing for all tasks, i.e. x4 | Eprime .dat and .txt |
| 9 | Physiology | Peripheral measure | Cardiac + respiratory traces for all runs, i.e. x6 | Philips "scanphyslog" |

## Data setup goals

For each dataset (i.e. for each subject) we have to:

1. Move all files into a machine readable directory structure
2. Rename all image files in this directory structure such that BIDS tags are findable
3. Convert data to BIDS:
  1. Run `bidsify` to convert the image data to BIDS (This includes conversion of PAR/REC to NIfTI using `dcm2niix`; this also includes anonymization, which doesn't work for some reason)
  2. Deface the T1w image using `pydeface`
  3. Run eprime conversion script to convert stimilus and response timings to BIDS (need to figure out this format)
  4. Run `scanphyslog2bids` (or Matlab script if needed) to convert physiology data to BIDS
4. Run the BIDS validator
6. Create summary tables and plots using `pybids`

## Required packages / software

- bidsify: https://github.com/NILAB-UvA/bidsify/tree/master/bidsify
- pydeface: https://github.com/poldracklab/pydeface
- fsl (required for pydeface): https://fsl.fmrib.ox.ac.uk/fsl/fslwiki
- bids_validator: https://github.com/bids-standard/bids-validator; https://bids-standard.github.io/bids-validator/
- convert-eprime: https://github.com/tsalo/convert-eprime
- scanphyslog2bids: https://github.com/lukassnoek/scanphyslog2bids

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import bidsify
import pydeface
import pandas as pd
import json
from bids_validator import BIDSValidator
from convert_eprime.convert import text_to_csv
from scanphyslog2bids.core import PhilipsPhysioLog

## Step 0: initialize variables, directories, filenames, etc


In [3]:
# Dependent on structure and filenames in current data on external drive
data_dir = '/Volumes/Stephan_WD/NEUFEPME_data/'
new_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data/NEUFEPME_data_new/'
org_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/'
new_org_data_dir = org_data_dir + 'new/'
bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_BIDS/'
bids_deriv_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_BIDS_derivatives/'
eprime_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_eprime/'

## Step 1+2: Move and rename all files into a machine readable and BIDS-ready directory structure


#### ONCE ONLY: copy all raw data, with exceptions, to new directory (leave raw data untouched!)


In [5]:
# Ignore files:
# - with 'FSL', this is a proxy for the FSL '.nii' file
# - with spaces (this is a proxy for the xml/rec files),
# - the PAR/RECs of T1w file
src = new_data_dir
dst = new_org_data_dir
ignore_pattern = shutil.ignore_patterns('*FSL*',
                                        '* *',
                                        '*T1W*.PAR',
                                        '*T1W*.REC')
shutil.copytree(src, dst, ignore=ignore_pattern)

'/Volumes/Stephan_WD/NEUFEPME_data_organised/new/'

#### ONCE ONLY: Rename files such that bidsify can read separate tags and convert correctly to BIDS format
See filenames in: "/Users/jheunis/Documents/PYTHON/rtme-fMRI/bidsify_test_2709/sub-01")


In [6]:
# This means removing the 'SENSE' text from the T1w file such that files with SENSE are all recognised as BOLD
os.chdir(new_org_data_dir)
t1w = glob.glob('*/*T1W*',recursive=True)
for fn in t1w:
    os.rename(fn, fn.replace('_SENSE', ''))

#### ONCE ONLY: Rename scanphyslog files based on time of acquisition and relation to functional scans

In [7]:
# Typically 10 files in each subject's 'scanphyslog' directory. If not, throw error.
scan_names = ['_emotion2', '_motor2', '_rest2', '_emotion1', '_motor1', '_rest1']
all_subs = next(os.walk(new_org_data_dir))[1]

print(all_subs)

for sub in all_subs:
    print(sub)
    os.chdir(new_org_data_dir + sub + '/scanphyslog')
    files = glob.glob('*')
    files.sort(key=os.path.getmtime, reverse=True)
    if len(files) != 10:
        print('ERROR: there are {} files. Not renaming files'.format(len(files)))
    else:
        for i, file in enumerate(files):
            if i < 6:
                new_file = os.path.splitext(file)[0] + scan_names[i] + os.path.splitext(file)[1]
                print(new_file)
                os.rename(file, new_file)
            else:
                print(file)

['sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
sub-021
SCANPHYSLOG20200210094910_emotion2.log
SCANPHYSLOG20200210094023_motor2.log
SCANPHYSLOG20200210093219_rest2.log
SCANPHYSLOG20200210092322_emotion1.log
SCANPHYSLOG20200210091439_motor1.log
SCANPHYSLOG20200210090629_rest1.log
SCANPHYSLOG20200210085925.log
SCANPHYSLOG20200210085911.log
SCANPHYSLOG20200210085612.log
SCANPHYSLOG20200210085551.log
sub-022
SCANPHYSLOG20200210114009_emotion2.log
SCANPHYSLOG20200210113155_motor2.log
SCANPHYSLOG20200210112346_rest2.log
SCANPHYSLOG20200210111334_emotion1.log
SCANPHYSLOG20200210110501_motor1.log
SCANPHYSLOG20200210105701_rest1.log
SCANPHYSLOG20200210104850.log
SCANPHYSLOG20200210104838.log
SCANPHYSLOG20200210104601.log
SCANPHYSLOG20200210104540.log
sub-023
SCANPHYSLOG20200210155308_emotion2.log
SCANPHYSLOG20200210154406_motor2.log
SCANPHYSLOG20200210153547_rest2.log
SCANPHYSLOG20200210152712_emotion1.log
SCANPHYSLOG2020

## Step 3.1: Convert image data to BIDS

Using `bidsify`

See `me_workflow.ipynb`


In [8]:
!cd /Volumes/Stephan_WD/NEUFEPME_data_organised/new && bidsify -v

Setting output-dir to /Volumes/Stephan_WD/NEUFEPME_data_organised/bids
Running bidsify with the following arguments:
	 directory=/Volumes/Stephan_WD/NEUFEPME_data_organised/new 
	 config=/Volumes/Stephan_WD/NEUFEPME_data_organised/new/config.yml 
	 out_dir=/Volumes/Stephan_WD/NEUFEPME_data_organised/bids 
	 validate=True

200303-10:25:11,429 nipype.utils INFO:
	 Running nipype version 1.2.3 (latest: 1.4.2)
/Users/jheunis/miniconda3/envs/rtme-fmri/lib/python3.7/site-packages/bidsify/main.py:173: UserWarning: The program 'bids-validator' was not found on your computer;
        setting the validate option to False
  warnings.warn(msg)
Converting data from sub-021 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.75775:0:0.00198914 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_d

Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.79219:0:0.00191241 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-021/NEUFEP-ME-021_NEUFEP-ME-021_NEUFEP-ME-rest2_SENSE_6_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.79219:0:0.00191241 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-021/NEUFEP-ME-021_NEUFEP-ME-021_NEUFEP-ME-rest2_SENSE_6_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.79219:0:0.00191241 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-021/NEUFEP-ME-021_NEUFEP-ME-021_NEUFEP-ME-rest2_SENSE_6_1_echo-3 (64x64x34x210)
Conversion required 5.546676 seconds (2.063248 for core code).
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/

Converting data from sub-022 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.7348:0:0.00196881 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-emotion1_SENSE_5_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.7348:0:0.00196881 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-emotion1_SENSE_5_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.7348:0:0.00196881 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-emotion1_SENSE_5_1_echo-3 (64x64x34x210)
Conversion required 5.762092 seconds (2.154735 for core code).
Compression will be 

Conversion required 6.074872 seconds (2.245057 for core code).
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-rest1_SENSE_3_1_echo-1.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/func/sub-022_task-rest_run-1_echo-1_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-rest1_SENSE_3_1_echo-1.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/func/sub-022_task-rest_run-1_echo-1_bold.nii'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-rest1_SENSE_3_1_echo-2.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/func/sub-022_task-rest_run-1_echo-2_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022_NEUFEP-ME-022_NEUFEP-ME-rest1_SENSE_3_1_echo-2.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/func/sub-022_task-r

Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/NEUFEP-ME-022^NEUFEP-ME-022_T1W_3D_TFE_2_1.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-022/anat/sub-022_T1w.nii'
Converting data from sub-023 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.74725:0:0.00186147 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-023/NEUFEP-ME-023_NEUFEP-ME-023_NEUFEP-ME-emotion1_SENSE_5_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.74725:0:0.00186147 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-023/NEUFEP-ME-023_NEUFEP-ME-023_NEUFEP-ME-emotion1_SENSE_5_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.74725:0:0.00186147 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_W

...

Conversion required 6.492298 seconds (2.059760 for core code).
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/NEUFEP-ME-030_NEUFEP-ME-030_NEUFEP-ME-rest1_SENSE_3_1_echo-1.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/func/sub-030_task-rest_run-1_echo-1_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/NEUFEP-ME-030_NEUFEP-ME-030_NEUFEP-ME-rest1_SENSE_3_1_echo-1.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/func/sub-030_task-rest_run-1_echo-1_bold.nii'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/NEUFEP-ME-030_NEUFEP-ME-030_NEUFEP-ME-rest1_SENSE_3_1_echo-2.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/func/sub-030_task-rest_run-1_echo-2_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/NEUFEP-ME-030_NEUFEP-ME-030_NEUFEP-ME-rest1_SENSE_3_1_echo-2.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-030/func/sub-030_task-r

Converting data from sub-031 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.73993:0:0.00203529 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-emotion1_SENSE_5_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.73993:0:0.00203529 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-emotion1_SENSE_5_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.73993:0:0.00203529 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-emotion1_SENSE_5_1_echo-3 (64x64x34x210)
Conversion required 4.697769 seconds (1.987963 for core code).
Compression will 

Philips Scaling Values RS:RI:SS = 0.73407:0:0.00206577 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-rest2_SENSE_6_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.73407:0:0.00206577 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-rest2_SENSE_6_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.73407:0:0.00206577 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-rest2_SENSE_6_1_echo-3 (64x64x34x210)
Conversion required 6.217240 seconds (2.033914 for core code).
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/NEUFEP-ME-031_NEUFEP-ME-031_NEUFEP-ME-rest1_SENSE_3_1_echo-1.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-031/func/sub-031_task-rest_run-1_echo-1_bold.json'
Renaming '/Volumes/Step

Converting data from sub-032 ...
Compression will be faster with 'pigz' installed http://macappstore.org/pigz/
Chris Rorden's dcm2niiX version v1.0.20190410  Clang9.0.0 (64-bit MacOS)
Done reading PAR header version 4.2, with 21420 slices
Philips Scaling Values RS:RI:SS = 0.78462:0:0.00167228 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-emotion1_SENSE_5_1_echo-1 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.78462:0:0.00167228 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-emotion1_SENSE_5_1_echo-2 (64x64x34x210)
Philips Scaling Values RS:RI:SS = 0.78462:0:0.00167228 (see PMC3998685)
Convert 1 DICOM as /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-emotion1_SENSE_5_1_echo-3 (64x64x34x210)
Conversion required 5.975177 seconds (2.062325 for core code).
Compression will 

Conversion required 6.387168 seconds (2.068134 for core code).
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-rest1_SENSE_3_1_echo-1.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-rest_run-1_echo-1_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-rest1_SENSE_3_1_echo-1.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-rest_run-1_echo-1_bold.nii'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-rest1_SENSE_3_1_echo-2.json' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-rest_run-1_echo-2_bold.json'
Renaming '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/NEUFEP-ME-032_NEUFEP-ME-032_NEUFEP-ME-rest1_SENSE_3_1_echo-2.nii' to '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-r

## Step 3.2: Deface T1w images

Using `pydeface`


In [11]:
# First run pydeface on all. This creates an extra T1w nifti image with defaced extension

# bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids'
# bids_deriv_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/derivatives'

# os.chdir(bids_data_dir)
# all_subs = next(os.walk(bids_data_dir))[1]
# print(all_subs)

# for i, sub in enumerate(all_subs):
#     anat_dir = os.path.join(bids_data_dir, sub, 'anat')
#     T1w_fn = os.path.join(anat_dir, sub+'_T1w.nii')
#     T1w_defaced_fn = os.path.join(anat_dir, sub+'_T1w_defaced.nii')
#     if not os.path.exists(T1w_defaced_fn):
#         print('{} - {}'.format(i, T1w_fn))
#         !cd $anat_dir && pydeface $T1w_fn
        
# # Then move both niftis to derivatives folder, remove the identifiable nifti, and rename defaced nifti to standard BIDS format
# os.chdir(bids_data_dir)
# all_subs = next(os.walk(bids_data_dir))[1]
# print(all_subs)
# for i, sub in enumerate(all_subs):
#     print('{} - {}'.format(i, sub))
#     anat_dir = os.path.join(bids_data_dir, sub, 'anat')
#     src = anat_dir    
#     dst = os.path.join(bids_deriv_data_dir, 'pydeface', sub, 'anat')
#     if not os.path.exists(dst):
#         shutil.copytree(src, dst)
#         T1w_fn = os.path.join(anat_dir, sub+'_T1w.nii')
#         T1w_defaced_fn = os.path.join(anat_dir, sub+'_T1w_defaced.nii')
#         os.remove(T1w_fn)
#         os.rename(T1w_defaced_fn, T1w_defaced_fn.replace('_defaced', ''))
#     else:
#         print('Already processed')

['sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
0 - sub-021
1 - sub-022
2 - sub-023
3 - sub-024
4 - sub-025
5 - sub-026
6 - sub-027
7 - sub-029
8 - sub-030
9 - sub-031
10 - sub-032


## Step 3.3: Convert Eprime data to BIDS

Use `neufepme_read_eprime.ipynb`, which uses `convert-eprime`

see: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/05-task-events.html

Perhaps write this into separate functions?


In [13]:
# import utilities as util
# pd.set_option('display.max_rows', None)

# bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids'
# bids_deriv_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/derivatives'

# paradigms = ['motor-', 'motor-imagine-', 'emotion-', 'emotion-imagine-']
# paradigm_nrs = ['-001', '-002', '-001', '-002']
# event_names = ['_task-motor_run-1_', '_task-motor_run-2_', '_task-emotion_run-1_', '_task-emotion_run-2_']
# keys = ['df0', 'df1', 'df2', 'df3']
# dct_df = dict.fromkeys(keys, None)

# # Loop through all subjects in BIDS folder
# os.chdir(bids_data_dir)
# all_subs = next(os.walk(bids_data_dir))[1]

# print(all_subs)

# for sub in all_subs:
#     # Find text files for each subject's four runs (see `paradigms` for run types)
#     # Create empty csv files (into which text file content is copied) and tsv files (for events in BIDS format)
#     sub_data_dir = os.path.join(eprime_data_dir, sub)
#     print(sub_data_dir)
#     txt_files = [None] * 4
#     csv_files = [None] * 4
#     event_files = [None] * 4
#     sub_nr = sub.split('-')[1]
#     for i, par in enumerate(paradigms):
#         txt_files[i] = os.path.join(sub_data_dir, 'neufepme-' + par + sub_nr + paradigm_nrs[i] + '.txt')
#         name, ext = os.path.splitext(txt_files[i])
#         csv_files[i] = name + '.csv'
#         event_files[i] = os.path.join(bids_data_dir, sub, 'func', sub+event_names[i]+'events.tsv')
#     # Read text files, convert to csv files, and cv files into dataframes 
#     all_dfs = {}
#     for i, fn in enumerate(txt_files):
#         text_to_csv(fn, csv_files[i])
#         df = pd.read_csv(csv_files[i])
#         all_dfs[paradigms[i]] = df

#     dct_df = util.create_event_files(all_dfs, paradigms, dct_df, sub)


#     ## FIX ALL THE DATAFRAMES TO MAKE THEM BIDS COMPATIBLE
#     # Drop the 'Rest' rows from df0, df1, df3
#     if not sub == 'sub-001':
#         for i in [0,1,3]:
#             df = dct_df[keys[i]]
#             idx = df[df['trial_type'] == 'Rest'].index
#             df.drop(idx , inplace=True)

#     # Update df2:
#     df = dct_df['df2']
#     # fill in missing values for sub-001:
#     if sub == 'sub-001':
#         df.loc[60, 'duration'] = 3.012
#         df.loc[60, 'response_time'] = 0.000
#     # 'button_pressed' = NaN ==> n/a; same rows 'response_time' ==> n/a
#     nan_rows = df[df['button_pressed'].isnull()]
#     df.loc[nan_rows.index, 'response_time'] = 'n/a'
#     df.loc[nan_rows.index, 'button_pressed'] = 'n/a'
#     # r = red = RIGHT ==> R; g = green = LEFT ==> L
#     r_rows = df[df['button_pressed'] == 'r']
#     df.loc[r_rows.index, 'button_pressed'] = 'R'
#     g_rows = df[df['button_pressed'] == 'g']
#     df.loc[g_rows.index, 'button_pressed'] = 'L'
#     # Fix onset times
#     df.loc[0, 'onset'] = 0.000
#     for i in df.index:
#         if i > 0:
#             df.loc[i, 'onset'] = np.round(df.loc[i-1, 'onset'] + df.loc[i-1, 'duration'], 3)

#     print(dct_df)

#     # Write to TSV files
#     for i, key in enumerate(keys):
#         if sub == 'sub-001':
#             if i==2:
#                 events = dct_df[key]
#                 events.to_csv(event_files[i], sep='\t', index=False)
#         else:   
#             events = dct_df[key]
#             events.to_csv(event_files[i], sep='\t', index=False)

['sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-029', 'sub-030', 'sub-031', 'sub-032', 'derivatives']


## Step 3.5: Add slice-timing information

In [14]:
bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids'
bids_deriv_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/derivatives'

# First run scripts to ensure that BIDS json data is correct
TR = 2
N_slices = 34
t_d = TR/N_slices
t_start = 0
t_stop = TR
slice_timing = np.arange(start=t_start, stop=t_stop, step=t_d).round(4).tolist()

os.chdir(bids_data_dir)
all_subs = next(os.walk(bids_data_dir))[1]
print(all_subs)

for sub in all_subs:
    os.chdir(os.path.join(bids_data_dir, sub, 'func'))
    json_files = glob.glob('*bold.json')
    if len(json_files) != 18:
        print('ERROR: there are {} BOLD json files, while there should be 18. Skipping subject: {}'.format(len(json_files), sub))
    else:
        for json_fn in json_files:
            with open(json_fn) as json_file:
                json_decoded = json.load(json_file)

            json_decoded["SliceTiming"] = slice_timing

            with open(json_fn, 'w') as json_file:
                json.dump(json_decoded, json_file)

['sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-029', 'sub-030', 'sub-031', 'sub-032', 'derivatives']


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/derivatives/func'

## Step 3.4: Convert physiology data to BIDS

Using `scanphyslog2bids`

See: https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/06-physiological-and-other-continuous-recordings.html


In [15]:
org_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/'
new_org_data_dir = org_data_dir + 'new/'
bids_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids'
bids_deriv_data_dir = '/Volumes/Stephan_WD/NEUFEPME_data_organised/bids/derivatives'

# Create folder into which scanphyslog2bids image outputs will be saved
bids_physio_dir = os.path.join(bids_deriv_data_dir, 'scanphyslog2bids')
if not os.path.exists(bids_physio_dir):
    os.makedirs(bids_physio_dir)

tasks = ['rest1', 'motor1', 'emotion1', 'rest2', 'motor2', 'emotion2']
tasks_bids = ['rest_run-1', 'motor_run-1', 'emotion_run-1', 'rest_run-2', 'motor_run-2', 'emotion_run-2']

os.chdir(bids_data_dir)
all_subs = next(os.walk(bids_data_dir))[1]

for sub in all_subs:
    # Create folder into which scanphyslog2bids image outputs will be saved, for current subject
    sub_physio_dir = os.path.join(bids_physio_dir, sub)
    if not os.path.exists(sub_physio_dir):
        os.makedirs(sub_physio_dir)
    # Initialise variables
    log_fns = [None] * len(tasks)
    log_renamed_fns = [None] * len(tasks)
    bids_fns = [None] * len(tasks)
    # Navigate to raw scanphyslog data for current subject
    scanphys_data_dir = os.path.join(new_org_data_dir, sub, 'scanphyslog')
    print(scanphys_data_dir)
    os.chdir(scanphys_data_dir)
    # For each scanphyslog file related to a specific functional run:
    for i, task in enumerate(tasks):
        fn = glob.glob('*'+task+'*')[0]
        log_fns[i] = os.path.join(scanphys_data_dir, fn)
        log_renamed_fns[i] = os.path.join(scanphys_data_dir, sub+'_task-'+tasks_bids[i]+'_physio.log')
        print(log_renamed_fns[i])
        # Copy and rename scanphyslog file to BIDS-compatible naming structure
        shutil.copy(log_fns[i], log_renamed_fns[i])
        # File to be converted
        log_file = log_renamed_fns[i]
        # Folder where the BIDSified data should be saved
        out_dir = os.path.join(bids_data_dir, sub, 'func')
        # Folder where some QC plots should be saved
        deriv_dir = sub_physio_dir
        # fmri_file is used to extract metadata, such as TR and number of volumes
        fmri_file = os.path.join(out_dir, sub+'_task-'+tasks_bids[i]+'_echo-2_bold.nii' ) 
        fmri_img = nib.load(fmri_file)
        n_dyns = fmri_img.shape[-1]
        tr = np.round(fmri_img.header['pixdim'][4], 3)
        # Create PhilipsPhysioLog object with info
        phlog = PhilipsPhysioLog(f=log_file, tr=tr, n_dyns=n_dyns, sf=500, manually_stopped=False)
        # Load in data, do some preprocessing
        phlog.load()
        # Try to align physio data with scan data, using a particular method
        # (either "vol_markers", "gradient_log", or "interpolate")
        phlog.align(trigger_method='interpolate')  # load and find vol triggers
        # Write out BIDS files - .tsv.gz and .json files
        phlog.to_bids(out_dir)
        # Optional: plot some QC graphs for alignment and actual traces
        phlog.plot_alignment(out_dir=deriv_dir)  # plots alignment with gradient
        phlog.plot_traces(out_dir=deriv_dir)  # plots cardiac/resp traces

/Volumes/Stephan_WD/NEUFEPME_data_organised/new/sub-021/scanphyslog
/Volumes/Stephan_WD/NEUFEPME_data_organised/new/sub-021/scanphyslog/sub-021_task-rest_run-1_physio.log


2020-03-03 12:14:20,211 [MainThread] [INFO   ]  Found end marker ('0020') at an offset of 22 samples relative to the end of the file.
2020-03-03 12:14:24,817 [MainThread] [INFO   ]  Trimmed off 59 samples from end of file based on the (absence) of a gradient.
2020-03-03 12:14:24,818 [MainThread] [INFO   ]  Found 210 triggers with a mean duration of 2.00300 (0.00100)!
2020-03-03 12:14:25,706 [MainThread] [INFO   ]  Saving BIDS data to /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-021/func/sub-021_task-rest_run-1_physio.tsv.
/Users/jheunis/miniconda3/envs/rtme-fmri/lib/python3.7/site-packages/scanphyslog2bids/core.py:570: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.tight_layout()
/Users/jheunis/miniconda3/envs/rtme-fmri/lib/python3.7/site-packages/scanphyslog2bids/core.py:581: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.savefig(base_name, dpi=100)


/Volumes/Stephan_WD/NEUFEPME_data_organised/new/sub-021/scanphyslog/sub-021_task-motor_run-1_physio.log


2020-03-03 12:14:31,680 [MainThread] [INFO   ]  Found end marker ('0020') at an offset of 53 samples relative to the end of the file.
2020-03-03 12:14:36,718 [MainThread] [INFO   ]  Trimmed off 60 samples from end of file based on the (absence) of a gradient.
2020-03-03 12:14:36,719 [MainThread] [INFO   ]  Found 210 triggers with a mean duration of 2.00300 (0.00100)!
2020-03-03 12:14:37,608 [MainThread] [INFO   ]  Saving BIDS data to /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-021/func/sub-021_task-motor_run-1_physio.tsv.


/Volumes/Stephan_WD/NEUFEPME_data_organised/new/sub-021/scanphyslog/sub-021_task-emotion_run-1_physio.log


...

2020-03-03 12:28:18,219 [MainThread] [INFO   ]  Found end marker ('0020') at an offset of 92 samples relative to the end of the file.
2020-03-03 12:28:23,245 [MainThread] [INFO   ]  Trimmed off 59 samples from end of file based on the (absence) of a gradient.
2020-03-03 12:28:23,246 [MainThread] [INFO   ]  Found 210 triggers with a mean duration of 2.00000 (0.00000)!
2020-03-03 12:28:24,356 [MainThread] [INFO   ]  Saving BIDS data to /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-motor_run-2_physio.tsv.


/Volumes/Stephan_WD/NEUFEPME_data_organised/new/sub-032/scanphyslog/sub-032_task-emotion_run-2_physio.log


2020-03-03 12:28:31,656 [MainThread] [INFO   ]  Found end marker ('0020') at an offset of 32 samples relative to the end of the file.
2020-03-03 12:28:37,013 [MainThread] [INFO   ]  Trimmed off 57 samples from end of file based on the (absence) of a gradient.
2020-03-03 12:28:37,014 [MainThread] [INFO   ]  Found 210 triggers with a mean duration of 2.00000 (0.00000)!
2020-03-03 12:28:37,947 [MainThread] [INFO   ]  Saving BIDS data to /Volumes/Stephan_WD/NEUFEPME_data_organised/bids/sub-032/func/sub-032_task-emotion_run-2_physio.tsv.


/Volumes/Stephan_WD/NEUFEPME_data_organised/new/derivatives/scanphyslog


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Stephan_WD/NEUFEPME_data_organised/new/derivatives/scanphyslog'

### Step 3.6 cleanup json bids files

## Step 4: Run BIDS validator

see: https://bids-standard.github.io/bids-validator/

In [6]:
# Check if possible to run validator on a full directory tree

# fn = '/Users/jheunis/Documents/PYTHON/rtme-fMRI/bidsify_test_2709/sub-01/anat/sub-01_T1w.nii'
# BIDSValidator().is_bids(fn)



## Step 6: Create summary tables and plots using `pybids`

See:
- https://bids-standard.github.io/pybids/
- https://github.com/bids-standard/pybids/blob/master/examples/pybids_tutorial.ipynb
- https://mybinder.org/v2/gh/bids-standard/pybids/master

this is also useful for de/constructing filenames, might be useful in earlier steps of this workflow already.

# Notes

## Data processing notes

Pybids gave `unknown locale UTF-8` issue, fixed by adding values to path variable upon conda env startup (and removing upon deactivation), see: https://coderwall.com/p/-k_93g/mac-os-x-valueerror-unknown-locale-utf-8-in-python

Need to build eprime code into a script, currently in a notebook

Remember for BIDS-validator:
- Delete T1w file and rename defaced file to original T1w name
- add events.tsv
- SliceTiming required for all functionals - need to find a better place to implement this in future. For now just a script to run through all files and update json with precalculated slice times based on known parameters. This is needed since Philips does not supply slice timing information in PAR/rec or DICOM.


## Data quality notes

#### Problem subjects: Eprime

- **sub-001** - Settings were not correct for saving OnsetToOnset time, Response time, Response, and other values. This means that missing data had to be completed. Steps done:
    - a) For event files for motor-run-1, motor-run-2 and emotion-run-2: copied event files from other subjects since the block paradigm stimulus timings are equal (to 2 decimal places).
    - b) For event file for emotion-run-2, only some values were missing, had to add some NaNs and replace this with average timing values from other stimulus timings from same run, same subject.


#### Problem subjects: EPI acquisition

- **sub-008** - last scan not finished (recon issue), need to rescan
- **sub-014** - last scan not finished (recon issue), need to rescan

#### Problem subjects: scanphyslog

UPDATE 13 January 2020:
*I incorrectly attributed the emotion1 scan restart problem to sub-015, it was actually sub-016 as confirmed on RIS system (see photos). This also explains "extra small file" previously mentioned in sub-016 dataset*

- **sub-015** - emotion1 scan was started twice, because heartrate sensor was not correctly recording. Second time was fine. need to inspect earlier scans for same issue. also, looks like scanphyslog didn't restart and kept loggin to same file, resulting in >12mb log file. need to inspect
- **sub-016** - All image data fine. Have to look at this data on RIS since scanphyslog has one extra small file which doesn't seem to fit

#### Problem subjects: T1w display

- **sub-017** - when displaying in Mango, top slices of brain are cut off, need to check if this is a viewer issue or acquisition issue. Need to check other subjects as well.

#### Steps done for now (8 Jan 2020):

- **sub-008** - deleted from organised data and BIDS data
- **sub-014** - deleted from organised data and BIDS data
- **sub-015** - removed from BIDS data, kept in organised data separate folder, scanphyslog renamed, need to inspect
- **sub-016** - removed from BIDS data, kept in organised data separate folder, scanphyslog NOT renamed, need to inspect

#### Steps to do (UPDATE 13 Jan 2020):

- **sub-015** - move back to BIDS and ensure everything is correctly named and converted
- **sub-016** - inspect everything, ensure this is correctly named and converted (ignore small file).

#### Steps done (UPDATE 30 Jan 2020):

- **sub-015** - move back to BIDS and ensure everything is correctly named and converted - DONE
- **sub-016** - inspect everything, ensure this is correctly named and converted - DONE
    - Small file deleted from organised data folder
    - script run to rename scanphyslog files x6
    - 
